# Lab 8 - Búsqueda por similitud usando feature extraction.

El objetivo de es implementar un sistema de búsqueda por similitud de imágenes usando feature extraction. Para ello, se utilizará un modelo pre-entrenado de clasificación de imágenes.

## Preparación del entorno.

Si no estamos parados en el repo, clonar y cd al repo. Esto nos permite usar el mismo notebook tanto local como en Google Colab.

In [ ]:
import os

REPO_NAME = "lab8"
if REPO_NAME not in os.getcwd():
  if not os.path.exists(REPO_NAME):
    !git clone https://github.com/FCEIA-AAII/{REPO_NAME}.git
  os.chdir(REPO_NAME)


Importar librerías

In [ ]:
import numpy as np
from pathlib import Path
import tensorflow as tf
from keras.layers import Input, Dense, GlobalMaxPooling2D
import matplotlib.pyplot as plt
from PIL import Image

Establecer GPU por defecto en caso de estar disponible.

In [ ]:
# Configurar para que TensorFlow utilice la GPU por defecto
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Configurar para que TensorFlow asigne memoria dinámicamente
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        # Especificar la GPU por defecto
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Manejar error
        print(e)

Cargar dataset:

In [ ]:
# Directorio de los datos
TRAIN_DATA_DIRECTORY = Path("dataset")

# Tamaño del lote (batch size)
BATCH_SIZE = 32

# Tamaño de las imágenes
IMAGE_HEIGHT = 224
IMAGE_WIDTH = 224

# Carga los datos.
dataset = tf.keras.utils.image_dataset_from_directory(
    TRAIN_DATA_DIRECTORY,
    label_mode=None,
    image_size=(IMAGE_HEIGHT, IMAGE_WIDTH),
    batch_size=BATCH_SIZE
)


Visualizar los datos:

In [ ]:
# Muestra algunas imágenes de ejemplo
plt.figure(figsize=(10, 10))
for images in dataset.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(f"Example: {i}")
    plt.axis("off")

Definimos un feature extractor.

In [ ]:
feature_extractor = tf.keras.applications.EfficientNetB0(input_shape=(IMAGE_HEIGHT, IMAGE_WIDTH,3 ),
                                                include_top=False,
                                                weights='imagenet',
                                                pooling="avg"
                                                )


Corremos el feature extractor sobre el dataset.

In [ ]:
features = feature_extractor.predict(dataset)

Asociamos cada feature con su respectiva imagen.

In [ ]:
# Obtener las imágenes correspondientes al dataset
image_paths = dataset.file_paths

# Asociar cada feature vector con su imagen correspondiente
image_features_mapping = dict(zip(image_paths, features))

Defino una función similitud, utilizo cosine_similarity:

In [ ]:
def cosine_similarity(v1, v2):
  return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

Ejemplo de cómo calcular similitud con `cosine_similarity`:

In [ ]:
path_1 = np.random.choice(image_paths)
path_2 = np.random.choice(image_paths)

v1 = image_features_mapping[path_1]
v2 = image_features_mapping[path_2]

print("Similitud:", cosine_similarity(v1, v2))

image_1 = plt.imread(path_1)
image_2 = plt.imread(path_2)

plt.figure(figsize=(10, 5))

# Subplot para la primera imagen
plt.subplot(1, 2, 1)
plt.imshow(image_1)
plt.title("Imagen 1")
plt.axis('off')

# Subplot para la segunda imagen
plt.subplot(1, 2, 2)
plt.imshow(image_2)
plt.title("Imagen 2")
plt.axis('off')

plt.show()

Completar el lab con la búsqueda por similitud.

Seleccionar una imagen al azar y buscar las 10 imágenes más similares.